# Trial Division

The most naive algorithm for integers factorization is [Trial Division][trial]: to find
the prime factors of $N$, we try to divide it by all prime numbers smaller than  $\sqrt{N}$.

To generate the prime numbers, I use the Sundaram sieve, but I exploit the iteration
interface of Julia to have a lazy implementation of it.


## Iteration Interface: a lazy implementation of Sundaram Sieve

Julia can be extended with new user defined types.
which can be integrated seamlessly into the language by implementing the
right [interfaces][interfaces].

Consider for example the _for_ loop

```
for i in obj
    do something with i
end
```

It can work with any object implementing the iteration interface, i.e. the methods
`start`, `next` and `done`.
That loop is syntactic sugar equivalent to

```
state = start(obj)
while !done(obj, state)
    i, state = next(obj, state)
end
```

To define a lazy version of the sundaram sieve, I create a new [composite type][composite-types],
containing the `is_prime` array of booleans (a bit array in this implementation), and the upper bound $ub$.

The type is parametric, since $ub$ can be if any type derived from `Integer`.

In [104]:
immutable SundaramSieve{T<:Integer}
    is_prime::BitArray
    ub::T
end

LoadError: invalid redefinition of constant SundaramSieve
while loading In[104], in expression starting on line 1

The iteration interface let me track the status of the iteration via the `state` object, which can be anything, and which is passed around by the three methods.

I use as state the last index checked by the sieve.

I start at 1, so `start` returns 1.

If $p$ is the current state, then `done` is true if there are no more primes between $p$ and $ub$. But to know that, the sieve must have run until $ub$. Thus, we maintain the sieving process one step beyond the prime returned at current iteration:
it computes until 7 but it returns 5; the next step it computes until 11 but it returns 7, and so on.

`next` receives the state, which allows to compute the prime it has to return, but it runs one step of the sieving until the next prime, or $ub$.

In [105]:
sundaram_sieve(n::Integer) = SundaramSieve(trues(n), div(n, 2)+1)

# the first state. In the state keep the last checked integer
Base.start(s::SundaramSieve) = 1

function Base.next(s::SundaramSieve, state)
    
    for i = state:s.ub
        step = i * 2 + 1
        for j=i+i*step:step:s.ub
            s.is_prime[j] = false
        end
        if s.is_prime[i]
            return (max(2, (state-1)*2+1), i+1)
        end
    end
    (max(2, (state-1)*2+1), s.ub)
end

Base.done(s::SundaramSieve, state) = state >= s.ub

done (generic function with 37 methods)

In [106]:
for i in sundaram_sieve(35)
    print("$i ")
end

2 3 5 7 11 13 17 19 23 29 31 

Another way of implementing lazy evaluated sequences are [coroutines][tasks].


## Coroutine

A coroutine is function whose evaluation can be suspended and resumed.
To create a coroutine, pass your function to the `Task` function.
Your function can produce data, and pass the control to the caller,
using the  `produce` function.

You can resume the coroutine execution using the `consume` function.
A Task implements also the iteration interface, so you can iterate over it
instead of explicitely calling `consume`.

In [107]:
function trial_division(n)
    function _f()
        if n < 2
            return
        end
        ub::Integer = ceil(sqrt(n)) + 1
    
        for prime in sundaram_sieve(ub)
            while n % prime == 0
                produce(prime)
                n = div(n, prime)
            end
        end
        if n > 1
            produce(n)
        end
    end
    Task(_f)
end

trial_division (generic function with 1 method)

In [117]:
for factor in trial_division(10)
    print("$factor ")
end

2 5 

[interfaces]: http://docs.julialang.org/en/release-0.4/manual/interfaces/
[composite-types]: http://docs.julialang.org/en/release-0.4/manual/types/#composite-types
[trial]: https://en.wikipedia.org/wiki/Trial_division
[tasks]: http://docs.julialang.org/en/release-0.4/manual/control-flow/#man-tasks